# Importing Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
import matplotlib.pyplot as plt
import numpy as np
import cv2



# Load and Preprocess the dataset
Note: Currently not including actual dataset in the repository to maintain privacy

In [ ]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array


IMG_SIZE = (128, 128)

def load_and_preprocess_images(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        image = load_img(img_path, target_size=IMG_SIZE)
        image = img_to_array(image) / 255.0  # to normalise between 0&1
        images.append(image)

    return np.array(images, dtype=np.float32)



synthetic_dataset = np.concatenate([synthetic_dataset_train, synthetic_dataset_test], axis=0)
real_used_dataset = load_and_preprocess_images("include dataset path here")
real_unused_dataset = load_and_preprocess_images("include dataset path here")


print(f"Synthetic: {synthetic_dataset.shape}")
print(f"Real Used: {real_used_dataset.shape}")
print(f"Real Unused: {real_unused_dataset.shape}")

X_train=synthetic_dataset

# Define the Autoencoder Structure

In [ ]:
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, LeakyReLU, Add, Input,
    Conv2DTranspose, GlobalAveragePooling2D, Dense, Reshape
)


def residual_block(x, filters):
    shortcut = x

    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)

    x = Conv2D(filters, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)

    x = Add()([x, shortcut])
    x = LeakyReLU(alpha=0.1)(x)
    return x

def resnet_autoencoder(input_shape=(128, 128, 3)):
    input_img = Input(shape=input_shape)

    # Encoder
    x = Conv2D(32, (3, 3), strides=2, padding='same')(input_img)
    x = residual_block(x, 32)

    x = Conv2D(64, (3, 3), strides=2, padding='same')(x)
    x = residual_block(x, 64)

    x = Conv2D(128, (3, 3), strides=2, padding='same')(x)
    x = residual_block(x, 128)

    x = Conv2D(256, (3, 3), strides=2, padding='same')(x)
    x = residual_block(x, 256)

    x = GlobalAveragePooling2D()(x)
    latent = Dense(512, activation='relu', name='latent')(x)

    # Decoder
    x = Dense(16 * 16 * 128, activation='relu')(latent)
    x = Reshape((16, 16, 128))(x)

    x = Conv2DTranspose(128, (3, 3), strides=2, padding='same')(x)
    x = residual_block(x, 128)

    x = Conv2DTranspose(64, (3, 3), strides=2, padding='same')(x)
    x = residual_block(x, 64)

    x = Conv2DTranspose(32, (3, 3), strides=2, padding='same')(x)
    x = residual_block(x, 32)

    output_img = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = models.Model(input_img, output_img)
    autoencoder.compile(optimizer='adam', loss='mae')
    return autoencoder


autoencoder_model = resnet_autoencoder()

encoder_model = models.Model(
    inputs=autoencoder_model.input,
    outputs=autoencoder_model.get_layer('latent').output
)


print("Autoencoder Summary:")
autoencoder_model.summary()

print("\nEncoder Summary:")
encoder_model.summary()


# Train the model

In [ ]:
autoencoder_model.fit(
    X_train, X_train,
    epochs=50,
    batch_size=32,

)

In [ ]:
autoencoder_model.save('/kaggle/working/autoencoder_model.keras')